In [ ]:
# install opencv-python
%pip install -U opencv-python
%pip install -U opencv-python-headless
%pip install pandas
%pip install matplotlib

### Retrieve video data from Kaggle

1. Go to the NFL Impact Detection Kaggle competition data [page](https://www.kaggle.com/competitions/nfl-impact-detection/data)
2. Under the `Data Explorer` section, download `train_labels.csv`
3. Click on `train` directory and pick any pairs of `endzone` and `sideline` videos of the same game-play (For this exercise, we'll be using `57583_000082_Endzone.mp4` and `57583_000082_Sideline.mp4`)
4. `train_labels.csv` and the videos will be downloaded as `zip` files, so we'll have to unzip them
5. Place the videos downloaded and `train_labels.csv` to the directory where the `fuse_and_visualize_multiview_impacts.ipynb` is located
6. run the following cell to unzip all the artifacts

In [ ]:
!unzip train_labels.csv.zip
!unzip 57583_000082_Sideline.mp4.zip
!unzip 57583_000082_Endzone.mp4.zip

## Import python packages required to run the fusion + video overlay process
Quick notes on some important python libraries:
* cv2: OpenCV python package that ingests, mainupulates and writes image and video data
* pandas: Python package for tabular data analysis
* numpy: Python package for scientific computing using arrays and matrices


In [ ]:
import os
import cv2
from IPython.display import Video, display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('mode.chained_assignment', None)

## Functions for deduplicating view and fusing events

* `prep_data(df)`: it takes a pandas dataframe as input and manipulates the dataframe to add a `game_play` column which will be in the format of `gameKey_playID` and returns the new dataframe with `game_play` column added
* `dedup_view(df, windows)`: This function takes a pandas dataframe with both sideline and endzone annotated impacts and windows (`int`) parameter, which is a +/- window of frames that will be used to match impacts identified from sideline and endzone views.
* `fuse_df(raw_df, windows)`: This is a wrapper function for `dedup_view` where it loops over all the unique `game_play`s and runs `dedup_view` on each unique `game_play` and concatenates all the results to a new pandas dataframe. This function takes the raw dataframe and windows (`int`) as parameters, which get fed into the `dedup_view` function

In [ ]:
def prep_data(df):
    df['game_play'] = df['gameKey'].astype('str') + '_' + df['playID'].astype('str').str.zfill(6)
    return df


def dedup_view(df, windows):
    # define view
    df = df.sort_values(by='frame')
    view_columns = ['frame', 'left', 'width', 'top', 'height', 'video']
    common_columns = ['game_play', 'label', 'view', 'impactType']
    label_cleaned = df[view_columns + common_columns]
    
    # rename columns
    sideline_column_rename = {col: 'Sideline_' + col for col in view_columns}
    endzone_column_rename = {col: 'Endzone_' + col for col in view_columns}
    sideline_columns = list(sideline_column_rename.values())
    
    # create two dataframes, one for sideline, one for endzone
    label_endzone = label_cleaned.query('view == "Endzone"')
    label_endzone.rename(columns=endzone_column_rename, inplace=True)
    label_sideline = label_cleaned.query('view == "Sideline"')
    label_sideline.rename(columns=sideline_column_rename, inplace=True)
    
    # prepare sideline labels
    label_sideline['is_dup'] = False
    for columns in sideline_columns:
        label_endzone[columns] = np.nan
    label_endzone['is_dup'] = False

    # iterrate endzone rows to find matches and dedup 
    for index, row in label_endzone.iterrows():
        player = row['label']
        frame = row['Endzone_frame']
        impact_type = row['impactType']
        sideline_row = label_sideline[(label_sideline['label'] == player) & (
                    (label_sideline['Sideline_frame'] >= frame - windows // 2) & (
                        label_sideline['Sideline_frame'] <= frame + windows // 2 + 1)) & (
                                                  label_sideline['is_dup'] == False) & (
                                                  label_sideline['impactType'] == impact_type)]

        if len(sideline_row) > 0:
            sideline_index = sideline_row.index[0]
            label_sideline['is_dup'].loc[sideline_index] = True

            for col in sideline_columns:
                label_endzone[col].loc[index] = sideline_row.iloc[0][col]
            label_endzone['is_dup'].loc[index] = True

    # calculate overlap perc
    not_dup_sideline = label_sideline[label_sideline['is_dup'] == False]
    final_output = pd.concat([not_dup_sideline, label_endzone])
    return final_output


def fuse_df(raw_df, windows):
    outputs = []
    all_game_play = raw_df['game_play'].unique()
    for game_play in all_game_play:
        df = raw_df.query('game_play ==@game_play')
        output = dedup_view(df, windows)
        outputs.append(output)

    output_df = pd.concat(outputs)
    output_df['gameKey'] = output_df['game_play'].apply(lambda x: x.split('_')[0]).map(int)
    output_df['playID'] = output_df['game_play'].apply(lambda x: x.split('_')[1]).map(int)

    return output_df


## Run the deduplication process

In [ ]:
# read the annotated impact data from train_labels.csv
ground_truth = pd.read_csv('train_labels.csv')

# prepare game_play column using pipe(prep_data) function in pandas then filter the dataframe for just rows with impacts
ground_truth = ground_truth.pipe(prep_data).query('impact == 1')

# loop over all the unique game_plays and deduplicate the impact results from sideline and endzone
fused_df = fuse_df(ground_truth, windows=30)


## Check what dataframes from above cell look like

In [ ]:
ground_truth.head()

In [ ]:
fused_df.query('gameKey == 57583').head()

## Define a function for overlaying helmet impacts from both views

* `get_video_and_metadata(vid_path)`: This function will take a the `video_path(str)` and return the video class as well as other metadata associated with the video, such as total_frame_number, width, height, frames per second (fps).

* `overlay_impacts(frame, fused_df, game_key, play_id, frame_cnt, h1)`: This is the main function that takes a frame and overlays impacts found from both views as well as individual views.

* `generate_impact_video(ez_vid_path, sl_vid_path, fused_df, output_path)`: This function is a wrapper for overlaying impacts. This will take the endzone video path, sideline video path, fused dataframe and name of the output `mp4` file.

In [ ]:
def get_video_and_metadata(vid_path):
    vid = cv2.VideoCapture(vid_path)
    total_frame_number = vid.get(cv2.CAP_PROP_FRAME_COUNT)
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = vid.get(cv2.CAP_PROP_FPS)
    return vid,total_frame_number, width, height, fps

def overlay_impacts(frame, fused_df, game_key, play_id, frame_cnt, h1):
    # look for duplicates
    #
    duplicates = fused_df.query(f"gameKey == {int(game_key)} and \
                               playID == {int(play_id)} and \
                               is_dup == True and \
                               Sideline_frame == @frame_cnt")
    frame_has_impact = False

    if len(duplicates) > 0:
        for duplicate in duplicates.itertuples(index=False):
            if frame_cnt == duplicate.Sideline_frame:
                frame_has_impact = True

            if frame_has_impact:
                cv2.rectangle(
                    frame,
                    (int(duplicate.Sideline_left), int(duplicate.Sideline_top)),
                    (int(duplicate.Sideline_left) + int(duplicate.Sideline_width), int(duplicate.Sideline_top) + int(duplicate.Sideline_height)),
                    (0,0,255), #RED boxes
                    thickness=3,
                             )

                cv2.rectangle(
                    frame,
                    (int(duplicate.Endzone_left), int(duplicate.Endzone_top)+ h1),
                    (int(duplicate.Endzone_left) + int(duplicate.Endzone_width), int(duplicate.Endzone_top) + int(duplicate.Endzone_height) + h1),
                    (0,0,255), #RED boxes
                    thickness=3,
                             )

                cv2.line(
                         frame, 
                         (int(duplicate.Sideline_left), int(duplicate.Sideline_top)),
                         (int(duplicate.Endzone_left), int(duplicate.Endzone_top) + h1),
                         (255, 255, 255), # WHITE lines
                         thickness=4
                        )

    else:
        # if no duplicates, look for sideline then endzone and add to the view
        #
        sl_impacts = fused_df.query(f"gameKey == {int(game_key)} and \
                                   playID == {int(play_id)} and \
                                   is_dup == False and \
                                   view == 'Sideline' and \
                                   Sideline_frame == @frame_cnt")
        if len(sl_impacts) > 0:
            for impact in sl_impacts.itertuples(index=False):
                if frame_cnt == impact.Sideline_frame:
                    frame_has_impact = True

                if frame_has_impact:
                    cv2.rectangle(
                        frame,
                        (int(impact.Sideline_left), int(impact.Sideline_top)),
                        (int(impact.Sideline_left) + int(impact.Sideline_width), int(impact.Sideline_top) + int(impact.Sideline_height)),
                        (0, 255, 255), #YELLOW BOXES
                        thickness=3,
                                 )

        ez_impacts = fused_df.query(f"gameKey == {int(game_key)} and \
                                   playID == {int(play_id)} and \
                                   is_dup == False and \
                                   view == 'Endzone' and \
                                   Endzone_frame == @frame_cnt")

        if len(ez_impacts) > 0:
            for impact in ez_impacts.itertuples(index=False):
                if frame_cnt == impact.Endzone_frame:
                    frame_has_impact = True

                if frame_has_impact:
                    cv2.rectangle(
                        frame,
                        (int(impact.Endzone_left), int(impact.Endzone_top)+ h1),
                        (int(impact.Endzone_left) + int(impact.Endzone_width), int(impact.Endzone_top) + int(impact.Endzone_height) + h1 ),
                        (0, 255, 255), #YELLOW BOXES
                        thickness=3,
                                 )
    return frame, frame_has_impact 

def generate_impact_video(ez_vid_path:str, 
                          sl_vid_path:str, 
                          fused_df:pd.DataFrame,
                          output_path:str,
                          freeze_impacts=True):

    #define video codec to be used for 
    VIDEO_CODEC = "MP4V"

    # parse game_key and play_id information from the name of the files
    #
    
    game_key = os.path.basename(ez_vid_path).split('_')[0] # parse game_key
    play_id = os.path.basename(ez_vid_path).split('_')[1] # parse play_id
    
    # define RGB values for different colors
    RED = (0, 0, 255)
    WHITE = (255, 255, 255)
    YELLOW = (0, 255, 255)
    
    # get metadata such as total frame number, width, height and frames per second (FPS) from endzone (ez) and sideline (sl) videos
    ez_vid, ez_total_frame_number, ez_width, ez_height, ez_fps = get_video_and_metadata(ez_vid_path)
    sl_vid, sl_total_frame_number, sl_width, sl_height, sl_fps = get_video_and_metadata(sl_vid_path)
    
    
    # define a video writer for the output video
    output_video = cv2.VideoWriter(output_path, 
                                   cv2.VideoWriter_fourcc(*VIDEO_CODEC), 
                                   ez_fps, (ez_width, ez_height+sl_height)) # stacking video vertically

    # find shorter video and use the total frame number from the shorter video for the output video
    total_frame_number = int(min(ez_total_frame_number, sl_total_frame_number))
    
    # iterate through each frame from endzone and sideline
    for frame_cnt in range(total_frame_number):
        frame_has_impact = False
        frame_near_impact = False
        
        # reading frames from both endzone and sideline
        ez_ret, ez_frame = ez_vid.read() 
        sl_ret, sl_frame = sl_vid.read()

        # creating strings to be added to the output frames
        #
        img_name = f"Game key: {game_key}, Play ID: {play_id}, Frame: {frame_cnt}"
        video_frame = f'{game_key}_{play_id}_{frame_cnt}'
        
        if ez_ret == True and sl_ret == True:
            
            h, w, c = ez_frame.shape
            h1,w1,c1 = sl_frame.shape

            if h != h1 or w != w1: # resize images if they're different
                ez_frame = cv2.resize(ez_frame,(w1,h1))

            frame = np.concatenate((sl_frame, ez_frame), axis=0) # stack the frames vertically
            
            
            frame, frame_has_impact = overlay_impacts(frame, fused_df, game_key, play_id, frame_cnt, h1)

            cv2.putText(frame, #image frame to be modified
                        img_name, #string to be inserted
                        (30, 30), #(x,y) location of the string
                        cv2.FONT_HERSHEY_SIMPLEX, #font
                        1, #scale
                        (255, 255, 255), #WHITE letters
                        thickness=2)
            
            cv2.putText(frame, #image frame to be modified
                        str(frame_cnt), #string to be inserted
                        (w1-75, h1-20), #(x,y) location of the string
                        cv2.FONT_HERSHEY_SIMPLEX, #font
                        1, #scale
                        (255, 255, 255), # WHITE letters
                        thickness=2)

            cv2.putText(frame, #image frame to be modified
                        str(frame_cnt), #string to be inserted
                        (w1-75, h1+h-20), #(x,y) location of the string
                        cv2.FONT_HERSHEY_SIMPLEX, #font
                        1, #scale
                        (255, 255, 255), # WHITE letters
                        thickness=2)

            output_video.write(frame)

            # Freeze for 60 frames on impacts
            if frame_has_impact and freeze_impacts:
                for _ in range(60):
                    output_video.write(frame)

        else: 
            break
        
        frame_cnt += 1
        
    output_video.release()

    return

In [ ]:
generate_impact_video('57583_000082_Endzone.mp4',
                      '57583_000082_Sideline.mp4',
                      fused_df,
                      'output.mp4'
                      )

## Download the video and visualize the results!

Finally, if you were able to run everything successfully, you should be able to see an `output.mp4` file in the left panel. Right click on the file and click on Download to your local computer. Then, just open the mp4 file and enjoy!